# Project 5 Report
#### Ben Raivel

In [5]:
%load_ext autoreload
import pandas as pd
import HashTable as ht
import serial
import multitasking
import multiprocessing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
files = ['rc/reddit_comments_2008.txt', 'rc/reddit_comments_2009.txt', 'rc/reddit_comments_2010.txt', 
                'rc/reddit_comments_2011.txt', 'rc/reddit_comments_2012.txt', 'rc/reddit_comments_2013.txt',
                'rc/reddit_comments_2014.txt', 'rc/reddit_comments_2015.txt']

years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]

In [3]:
multi_table = ht.HashTable()

file = open(files[0], 'r')

block_gen = multitasking.gen_block(file)

pool = multiprocessing.Pool()

sub_tables = pool.imap(multitasking.m_count_words, block_gen)

# for each table returned by pool
for t in sub_tables:

    # combine with the main table
    multi_table.combine(t)

print(multi_table.to_df(2008).head())

64116, 128, 0, 1.1004
64126, 256, 82, 0.0138
64126, 512, 145, 0.0158
64126, 1024, 310, 0.0298
64126, 2048, 674, 0.0436
64128, 256, 72, 0.0030
64128, 512, 135, 0.0051
64128, 1024, 304, 0.0108
64128, 2048, 679, 0.0307
64126, 4096, 1584, 0.0926
64128, 4096, 1554, 0.0830
64127, 256, 110, 0.0039
64127, 512, 173, 0.0059
64127, 1024, 379, 0.0357
64127, 2048, 792, 0.0898
64126, 8192, 3656, 0.2546
64127, 4096, 1711, 0.1359
64128, 8192, 3572, 0.2390
64127, 8192, 3931, 0.2131
64129, 256, 74, 0.0033
64129, 512, 137, 0.0054
64129, 1024, 312, 0.0120
64129, 2048, 657, 0.0236
64129, 4096, 1502, 0.0979
64129, 8192, 3431, 0.1712
64126, 16384, 8699, 0.5103
64128, 16384, 8699, 0.4800
64127, 16384, 9611, 0.4811
64129, 16384, 8488, 0.3803
64126, 32768, 23051, 1.1136
64128, 32768, 22296, 1.1908
64127, 32768, 24273, 1.1066
64129, 32768, 21847, 0.9303
64128, 65536, 62422, 2.7591
64129, 65536, 64068, 2.5582
64126, 65536, 68464, 2.9126
64127, 65536, 68122, 2.8474
64128, 131072, 210376, 8.2866
64129, 131072, 2104

In [6]:
my_df = pd.DataFrame({'A': [0, 1, 2, 3, 4], 'B': [5, 6, 7, 8, 9], 'C': ['a', 'b', 'c', 'd', 'e']})

print(my_df)

print(my_df.replace(0))